In [4]:
import requests

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(url).text

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(website_url,'lxml')

In [8]:
my_table = soup.find('table',{'class':'wikitable sortable'})

In [9]:
links = my_table.findAll('td')
# str(links[5])[4:-6]
links[0]

<td>M1A
</td>

In [10]:
f'number of rows in our table is: {len(links)/3}'

'number of rows in our table is: 180.0'

In [12]:
Postcodes = [] 
Boroughs = [] 
neighborhoods = []

i = 0

for word in links:
  if i%3 == 0:
    Postcodes.append(word.get_text().strip())
    
  elif i%3 == 1:
    Boroughs.append(word.get_text().strip())
  
  elif i%3 == 2:
    neighborhoods.append(word.get_text().strip())
  i+=1

In [13]:
len(neighborhoods) == len(Boroughs) == len(Postcodes)

True

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame()
df['Postalcodes'] = Postcodes
df['Borough'] = Boroughs
df['neighborhood'] = neighborhoods
df[df['Postalcodes']== 'M1R']

,Postalcodes,Borough,neighborhood
108,M1R,Scarborough,"Wexford, Maryvale"


In [16]:
indexes = []
for index, row in df.iterrows():
  if row['Borough'] == 'Not assigned':
    indexes.append(index)
    continue
  elif row['neighborhood'] == 'Not assigned':
    row['neighborhood'] == row['Borough']
df.head()

,Postalcodes,Borough,neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [17]:
df.drop(indexes, inplace=True)
df.head()

,Postalcodes,Borough,neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [18]:
df1 = df.groupby('Postalcodes')['neighborhood'].apply(', '.join).reset_index()
df1.head()

,Postalcodes,neighborhood
0,M1B,"Malvern, Rouge"
1,M1C,"Rouge Hill, Port Union, Highland Creek"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [19]:
df = pd.merge( df[['Postalcodes','Borough']], df1[['Postalcodes','neighborhood']], on='Postalcodes')
df.head()

,Postalcodes,Borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [20]:
df.shape

(103, 3)

In [21]:
!wget -O geospatial_data.csv 'http://cocl.us/Geospatial_data'

--2020-12-12 06:41:00--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.194, 169.63.96.176
Connecting to cocl.us (cocl.us)|169.63.96.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-12-12 06:41:01--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.63.96.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-12-12 06:41:02--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-12-12 06:41:02--  https://ibm.box.com/public/static/9afzr83p

In [22]:
geospatial_data = pd.read_csv('geospatial_data.csv')
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
geospatial_data = geospatial_data.rename(columns={"Postal Code": "Postalcodes"}, errors="raise")
geospatial_data.head()

,Postalcodes,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
df = df.merge(geospatial_data, on='Postalcodes')
                                                        
df.head()

,Postalcodes,Borough,neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [25]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [26]:
import json
!pip install geopy 
from geopy.geocoders import Nominatim
!pip install folium==0.5.0
import folium

In [27]:
address = 'Toronto'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [28]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto